In [1]:
!pip install -q git+https://github.com/tensorflow/examples.git
!pip install tensorflow_datasets
!pip install matplotlib
!pip install pillow

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt
import os

from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

from PIL import Image

import numpy as np

predicted_path = '../Data/Dataset/Predicted_Depth/'
original_path = '../Data/Dataset/Original_Depth/'
models_path = '../Models/GANs/'
model_name = 'Apr-27-2021_error_generator'
testing_append = 'Testing/'
training_append = 'Training/'

In [3]:
# DATASET PARAMETERS
SEED = 123
BATCH_SIZE = 32
BUFFER_SIZE = 1000
IMG_SIZE = 512
# DATASET CONFIGS
ground_truth_path = original_path + testing_append
# training_set_path = dataset_path
print(type(ground_truth_path))
# !nvidia-smi --query-gpu=memory.used --format=csv
ground_truth_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ground_truth_path,label_mode=None,
    class_names=None, color_mode='grayscale', batch_size=BATCH_SIZE, image_size=(IMG_SIZE,
    IMG_SIZE), shuffle=False, seed=SEED, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False
)
# !nvidia-smi --query-gpu=memory.used --format=csv
ground_truth_dataset = ground_truth_dataset.unbatch()
# ground_truth_dataset = ground_truth_dataset.shuffle(BUFFER_SIZE, seed=SEED)
ground_truth_dataset = ground_truth_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(ground_truth_dataset)

testing_data_path = predicted_path + testing_append
print(type(testing_data_path))
testing_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    testing_data_path,label_mode=None,
    class_names=None, color_mode='grayscale', batch_size=BATCH_SIZE, image_size=(IMG_SIZE,
    IMG_SIZE), shuffle=False, seed=SEED, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False
)
testing_dataset = testing_dataset.unbatch()
# testing_dataset = testing_dataset.shuffle(BUFFER_SIZE, seed=SEED)
testing_dataset = testing_dataset.batch(BATCH_SIZE, drop_remainder=True)


<class 'str'>
Found 230 files belonging to 1 classes.
<BatchDataset shapes: (32, 512, 512, 1), types: tf.float32>
<class 'str'>
Found 230 files belonging to 1 classes.


In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
  
    
def normalize(input_image: tf.Tensor) -> tuple:
    """Rescale the pixel values of the images between 0.0 and 1.0
    compared to [0,255] originally.

    Parameters
    ----------
    input_image : tf.Tensor
        Tensorflow tensor containing an image of size [SIZE,SIZE,3].
    input_mask : tf.Tensor
        Tensorflow tensor containing an annotation of size [SIZE,SIZE,1].

    Returns
    -------
    tuple
        Normalized image and its annotation.
    """
    input_image = (tf.cast(input_image, tf.float32))
    input_image = (input_image - tf.math.reduce_min(input_image))
    input_image = input_image / tf.math.reduce_max(input_image)
    return input_image


In [6]:
def xyzmwrite(path, x, y, z, r, g, b, m):
    w, h = r.shape # This seems to be a problem with the naming scheme in the original MATLAB xyzmwrite, or I may just be dumb

    xyz = np.zeros((w*h, 3))

    xm1 = np.array(np.reshape(x, w*h, order = 'F'))
    ym1 = np.array(np.reshape(y, w*h, order = 'F'))
    zm1 =  np.array(np.reshape(z, w*h, order = 'F'))
    xyz[:, 0] = xm1
    xyz[:, 1] = ym1
    xyz[:, 2] = zm1
    xyz = np.reshape(xyz.T, w*h*3, order = 'F')

    rgb = np.zeros((w*h, 3))
    r1 = np.array(np.reshape(r, w*h, order = 'F'))
    g1 = np.array(np.reshape(g, w*h, order = 'F'))
    b1 =  np.array(np.reshape(b, w*h, order = 'F'))
    rgb[:, 0] = r1
    rgb[:, 1] = g1
    rgb[:, 2] = b1
    rgb = np.reshape(rgb.T, w*h*3, order = 'F')

    m = np.reshape(m, w*h, order = 'F')

    try:
        os.remove(path)
    except OSError:
        pass

    with open(path, 'wb') as xyzm_file:
        xyzm_file.write(b'image size width x height = %d x %d'%(w, h))
        xyzm_file.write(xyz.astype(np.float32))
        xyzm_file.write(rgb.astype(np.uint8))
        xyzm_file.write(m.astype(np.uint8))

    return None

def xyzmread(path):
    # Find height and width of data from header
    file_object = open(path, 'rb')                                              # Open file
    file_header = file_object.read(100).decode('ascii', errors = 'ignore')      # Get header as ASCII string
    file_header = ''.join(filter(lambda x: x in string.printable, file_header)) # Filter non-printable characters out
    file_header_info = file_header.split('=')                                        
    file_header_info = file_header_info[1].split('x')
    width = int(file_header_info[0])
    height_string = file_header_info[1]
    valid_height_string = ''
    for i in range(1,len(height_string)):                                       # Range 1-len(height_string) ignores the space after the 'x'
        if (ord(height_string[i]) < 48 or ord(height_string[i]) > 57):          # Do not add to valid string if values lie outside '0' - '9'
            break
        valid_height_string = valid_height_string + height_string[i]
    height = int(valid_height_string)

    # Ignore header in data
    # Additional 1's added to reflect the removal of characters in .split() method
    header_length = len(file_header.split('x')[0]) + 1 + len(file_header.split('x')[1]) + 1 + len(valid_height_string) + 1
    file_object.seek(header_length, 0)
    # Read in data
    xyz_data = []
    rgb_data = []
    m_data = []
    for i in range(0, (width*height*3)):
        (num,) = struct.unpack('f', file_object.read(4))
        xyz_data.append(num)

    for j in range(0, (width*height*3)):
        (num,) = struct.unpack('>H', b'\x00' + file_object.read(1))
        rgb_data.append(num)
    
    for k in range(0, (width*height)):
        (num,) = struct.unpack('>H', b'\x00' + file_object.read(1))
        m_data.append(num)

    xyz_data = np.reshape(xyz_data, (3, width*height), order = 'F')
    x_data = xyz_data[0,:]
    x_data = np.reshape(x_data, (width, height), order = 'F')
    y_data = xyz_data[1,:]
    y_data = np.reshape(y_data, (width, height), order = 'F')
    z_data = xyz_data[2,:]
    z_data = np.reshape(z_data, (width, height), order = 'F')

    rgb_data = np.reshape(rgb_data, (3, width*height), order = 'F')
    r_data = rgb_data[0,:]
    r_data = np.reshape(r_data, (width, height), order = 'F')
    g_data = rgb_data[1,:]
    g_data = np.reshape(g_data, (width, height), order = 'F')
    b_data = rgb_data[2,:]
    b_data = np.reshape(b_data, (width, height), order = 'F')

    m_data = np.reshape(m_data, (width, height), order = 'F')

    return x_data, y_data, z_data, r_data, g_data, b_data, m_data

def xyzmshow2d(x, y, z, r, g, b, m):
    print('Preparing 2D...')
    # plt.ion()
    # plt.show()

    fig, axs = plt.subplots(3,3)
    axs[0, 0].imshow(x)
    axs[0, 0].set_title('X')
    axs[0, 1].imshow(y)
    axs[0, 1].set_title('Y')
    axs[0, 2].imshow(z)
    axs[0, 2].set_title('Z')
    axs[1, 0].imshow(r)
    axs[1, 0].set_title('R')
    axs[1, 1].imshow(g)
    axs[1, 1].set_title('G')
    axs[1, 2].imshow(b)
    axs[1, 2].set_title('B')
    axs[2, 0].imshow(m)
    axs[2, 0].set_title('M')
    fig.delaxes(axs[2,1])
    fig.delaxes(axs[2,2]) 

    # plt.draw()
    # plt.pause(0.001)
    print('Displaying 2D...')
    plt.show()
    return

In [7]:
def display(display_list):
    plt.figure()

    title = ['Input Image', 'Corrected Depth']

#     for i in range(len(display_list)):
# #         plt.subplot(1, len(display_list), i+1)
#         plt.title(title[i])
# #         data = data.reshape((display_list[i].shape[0], display_list[i].shape[1], 1)
#         plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[0]))
#         d = display_list[1]
#         plt.imshow(d[1, :, :, 0]/255.0, cmap = 'gray')
#         # plt.axis('off')
#     print(np.max(display_list[0]))
#     print(np.min(display_list[0]))
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[0]))
    d = display_list[1]
#     print(d.shape)
#     print(np.max(d))
#     print(np.min(d))
    plt.show()
    plt.figure()
    plt.imshow(d * 127.5 + 127.5, cmap = 'gray')
    plt.show()
    

def show_predictions(dataset=None, num=1):
    if dataset:
        for image, depth in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], depth[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask, create_mask(model.predict(sample_image[tf.newaxis, ...]))])
        
def evaluate_predictions(dataset=None, num=1):
    if dataset:
        for image in dataset.take(num):
            corr_depth = model.predict(image)
#             mask = create_mask(pred_mask)
#             decoded_depth = scdDecode(image[0], mask, depth[0], verbose = True)
            
#             print('Actual', 'Min Depth:', np.min(depth[0]), 'Max Depth:', np.max(depth[0]))
#             print('Predicted', 'Min Depth:', np.min(decoded_depth), 'Max Depth:', np.max(decoded_depth))
            display([image[0,:,:,:], corr_depth[0,:,:,:]])
            
def evaluate_all_data(dataset=None, save_images=False, save_testing=False):
      
    if dataset:
        num_images = 0
        total_sum_squared_diff = 0
        for batch in dataset:
            images = batch
#             depths = batch[1]
            
            for i in range(0, len(images)):
                image = images[i,:,:,:]
#                 depth = depths[i]
#                 print(image.shape)
                image = tf.reshape(image, [1, IMG_SIZE, IMG_SIZE, 1]) # -1 to get None first dimensions         
                
                corrected_depth = model.predict(image)
#                 mask = create_mask(corrected_depth)
                corrected_depth = tf.squeeze(corrected_depth,0)
                image = tf.squeeze(image,0)
                if save_images:
                    # Save Predicted Depth
                    corrected_depth_image = tf.keras.preprocessing.image.array_to_img(corrected_depth)
                    plt.imsave('../Data/Dataset/Analysis_Outputs/Corrected_Depth_Classic_GAN/Testing/' + str(num_images + i + 1) + '_SCD_Corrected_Depth.png', corrected_depth_image)
                    # Save Original Depth input
                    original_depth_image = tf.keras.preprocessing.image.array_to_img(image)
                    plt.imsave('../Data/Dataset/Analysis_Outputs/Original_Depth/Testing/' + str(num_images + i + 1) + '_SCD_Original_Depth.png', original_depth_image)
                    
                    x = np.linspace(0, 512, 512)
                    y = x
                    xx, yy = np.meshgrid(x, y)
                    xyzmwrite('../Data/Dataset/Analysis_Outputs/Corrected_Depth_Classic_GAN/Testing/' + str(num_images + i + 1) + '_SCD_Corrected_Depth.xyzm', xx, yy, corrected_depth_image, np.zeros([512, 512]), np.zeros([512, 512]), np.zeros([512, 512]), np.ones([512, 512]))

                # Calc RMS error for whole dataset
                corrected_depth = corrected_depth*127.5 + 127.5
                image = image*127.5 + 127.5
                diff = corrected_depth - image
#                 print(np.max(diff) - np.min(diff))
                if save_images:
                    diff_image = tf.keras.preprocessing.image.array_to_img(diff)
                    plt.imsave('../Data/Dataset/Analysis_Outputs/Diff_Image_Classic_GAN/Testing/' + str(num_images + i + 1) + '_SCD_Diff_Image.png', diff_image)
                squared_diff = np.power(diff, 2)
                sum_squared_diff = np.sum(squared_diff)
                total_sum_squared_diff = total_sum_squared_diff + sum_squared_diff
                
            num_images = num_images + i + 1
            
        num_points = IMG_SIZE * IMG_SIZE * num_images
        rms_error = np.sqrt(total_sum_squared_diff / num_points)     
        print('Number of Images Tested: ', num_images)
        print('RMS Error: ', rms_error)

    
                
def calc_rms_error(true_depth, pred_depth):
    diff = pred_depth - true_depth
    squared_diff = np.power(diff, 2)
    sum_squared_diff = np.sum(squared_diff)
    
    num_points = pred_depth.size
    
    return np.sqrt(sum_squared_diff / num_points)

def calc_avg_abs_error(true_depth, pred_depth):
    abs_error = np.abs(pred_depth - true_depth)
    avg_abs_error = np.sum(abs_error) / pred_depth.size
    
    return avg_abs_error
    


In [8]:
model = keras.models.load_model(models_path + model_name, compile=False)
ground_truth_dataset = ground_truth_dataset.map(normalize) # Normalize the images to [0, 1]
testing_dataset = testing_dataset.map(normalize)

In [9]:
# evaluate_predictions(testing_dataset, 10)

In [10]:
evaluate_all_data(ground_truth_dataset, save_images=True, save_testing=True)

Number of Images Tested:  224
RMS Error:  12.174309938020574
